In [1]:
# UPGRADE XLSXX
import pandas as pd
import os
from datetime import datetime

FILE = 'Finance_Ledger.xlsx'

def upgrade_ledger():
    if not os.path.exists(FILE):
        print(f"❌ '{FILE}' not found. Please run reset_system.py instead.")
        return

    print(f"🔄 Upgrading '{FILE}' to Version 3 format...")
    
    # --- 1. Load Existing Data (Handle missing sheets safely) ---
    try:
        df_inv = pd.read_excel(FILE, sheet_name='Invoices')
    except:
        df_inv = pd.DataFrame()
        print("   - Created missing 'Invoices' sheet.")

    try:
        df_pay = pd.read_excel(FILE, sheet_name='Payments')
    except:
        df_pay = pd.DataFrame()
        print("   - Created missing 'Payments' sheet.")

    # --- 2. Add Missing Columns to INVOICES ---
    required_inv_cols = ['Invoice_No', 'Date', 'Client', 'Project_Name', 'Total_Amount', 'PDF_File', 'Business_Unit']
    for col in required_inv_cols:
        if col not in df_inv.columns:
            df_inv[col] = "" # Add empty column
            print(f"   + Added column '{col}' to Invoices")

    # --- 3. Add Missing Columns to PAYMENTS ---
    required_pay_cols = ['Payment_ID', 'Invoice_Ref', 'Amount_Received', 'Method', 'Proof_File', 'Payment_Date', 'Entry_Date']
    for col in required_pay_cols:
        if col not in df_pay.columns:
            if col == 'Payment_Date':
                # Migration: Copy old 'Date' column if it exists
                if 'Date' in df_pay.columns:
                    df_pay['Payment_Date'] = df_pay['Date']
                else:
                    df_pay['Payment_Date'] = datetime.today()
            elif col == 'Entry_Date':
                # Migration: Backfill entry date as today
                df_pay['Entry_Date'] = datetime.now()
            else:
                df_pay[col] = ""
            print(f"   + Added column '{col}' to Payments")

    # --- 4. Create/Reset MASTER_LEDGER Sheet ---
    # We don't need to migrate this, we just ensure the tab exists. 
    # The App will auto-fill it on next save.
    df_master = pd.DataFrame(columns=['Date', 'Description', 'Debit', 'Credit', 'Type', 'Business_Unit', 'Entry_Date'])
    print("   - Reset/Created 'Master_Ledger' sheet structure.")

    # --- 5. Save Everything Back ---
    with pd.ExcelWriter(FILE, engine='openpyxl') as writer:
        df_inv.to_excel(writer, sheet_name='Invoices', index=False)
        df_pay.to_excel(writer, sheet_name='Payments', index=False)
        df_master.to_excel(writer, sheet_name='Master_Ledger', index=False)

    print("\n✅ Upgrade Complete! Your Excel file is ready for the new App.")

if __name__ == "__main__":
    upgrade_ledger()


🔄 Upgrading 'Finance_Ledger.xlsx' to Version 3 format...
   - Reset/Created 'Master_Ledger' sheet structure.

✅ Upgrade Complete! Your Excel file is ready for the new App.


In [11]:
import pandas as pd
import os

FILE = 'Finance_Ledger.xlsx'

def reset_ledger():
    print(f"⚠️  WARNING: This will DELETE '{FILE}' and all its data.")
    confirm = input("Type 'DELETE' to confirm: ")
    
    if confirm == "DELETE":
        if os.path.exists(FILE):
            os.remove(FILE)
            print("🗑️  Old file deleted.")
        
        # 1. Invoices Sheet (Strict Order)
        df_inv = pd.DataFrame(columns=[
            'Invoice_No', 'Date', 'Entry_Date', 'Client', 'Project_Name', 
            'Total_Amount', 'PDF_File', 'Business_Unit'
        ])
        
        # 2. Payments Sheet (Strict Order)
        df_pay = pd.DataFrame(columns=[
            'Payment_ID', 'Invoice_Ref', 'Amount_Received', 'Method', 
            'Proof_File', 'Payment_Date', 'Entry_Date'
        ])
        
        # 3. Master Ledger Sheet
        df_master = pd.DataFrame(columns=[
            'Transaction_Date', 'System_Entry_Date', 'Description', 'Debit', 'Credit', 
            'Balance', 'Type', 'Business_Unit'
        ])

        with pd.ExcelWriter(FILE, engine='openpyxl') as writer:
            df_inv.to_excel(writer, sheet_name='Invoices', index=False)
            df_pay.to_excel(writer, sheet_name='Payments', index=False)
            df_master.to_excel(writer, sheet_name='Master_Ledger', index=False)
            
        print(f"✅ Success! Created fresh '{FILE}' with Correct Columns.")
    else:
        print("❌ Reset cancelled.")

if __name__ == "__main__":
    reset_ledger()

⚠️  WARNING: This will DELETE 'Finance_Ledger.xlsx' and all its data.


Type 'DELETE' to confirm:  DELETE


🗑️  Old file deleted.
✅ Success! Created fresh 'Finance_Ledger.xlsx' with Correct Columns.
